In [1]:
%pip install nltk
%pip install tweepy
%pip install pandas
%pip install wordcloud
%pip install textblob
%pip install seaborn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import tweepy
import json
import csv
import sys
import time
import re
import warnings
import string
import pandas as pd
import ast
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import Stream
from wordcloud import WordCloud
from textblob import TextBlob
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
warnings.filterwarnings('ignore')

[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


In [285]:
ckey = 'o7qidRODRWm5u07PtAWrCpSqf'
csecret = '1Yn60OYMJl5kRCSM0XQ4Lskxh6hjM4VMz51OdsS6dM9DQuQ4MV'
atoken = '1315792402234441730-Dt7AydgVGYI5VN9RRQfnR8Endh0WAc'
asecret = 'jhfiDfB53Z27EqHfvfKbNdo0w2SWLqyyoAkILqgSoaGMK'

OAUTH_KEYS = {'consumer_key':ckey, 'consumer_secret':csecret, 'access_token_key':atoken, 'access_token_secret':asecret}
auth = tweepy.OAuthHandler(OAUTH_KEYS['consumer_key'], OAUTH_KEYS['consumer_secret'])

api = tweepy.API(auth)

In [286]:
def Ceviri(DataSet):
    print ("Tweetler Çeviriliyor...")
    
    for yazi in range(len(DataSet["Tweet"])):
        Ceviri_Blob=TextBlob(DataSet["Tweet"][yazi])
        Eng_Blob=Ceviri_Blob.translate(from_lang='tr', to='en')  
        DataSet['Tweet_Ceviri'][yazi]=Eng_Blob
        
    print ("Tweetler Çevirildi!")
    return DataSet

In [287]:
def TweetTemizle(Dataset):
    print ("Tweetler Temizleniyor...")

    pattern = re.compile('http[s]?://(?:[A-Za-z0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+|#[A-Za-z0-9]+|$[A-Za-z0-9]+|@[A-Za-z0-9_]+|[,.^_$*%-;鶯!?’"“”:=|…]+|\n')
    for i in range(len(Dataset["Tweet_Degistirilmis"])):        
        Dataset["Tweet_Degistirilmis"][i]=pattern.sub(' ', Dataset["Tweet_Degistirilmis"][i])
        
    print ("Tweetler Temizlendi!")
    Ceviri(Dataset)
    return Dataset

In [288]:
def KelimeAyristirma(DataSet):
    data_test = DataSet.copy()
    stop = stopwords.words('english')
    data_test["Tweet_Degistirilmis"] = data_test["Tweet_Degistirilmis"].str.lower().str.split()
    data_test["Tweet_Degistirilmis"] = data_test["Tweet_Degistirilmis"].apply(lambda x: [item for item in x if item not in stop])

def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity
def getPolarity(text):
    return TextBlob(text).sentiment.polarity 

In [289]:
def toDataFrame(tweets):
    print ("Veriler Kayıt Ediliyor...")
    # COnvert to data frame    
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)
    pd.set_option('display.max_colwidth', None)
    DataSet = pd.DataFrame()


    DataSet['Tweet_Id'] = [tweet.id for tweet in tweets]
    DataSet['Tweet'] = [tweet.text for tweet in tweets]
    DataSet['Tweet_Degistirilmis'] = [tweet.text for tweet in tweets]
    DataSet['Tweet_Ceviri']=[tweet.text for tweet in tweets]
    DataSet['Tweet_Tarih'] = [tweet.created_at for tweet in tweets]
    DataSet['Kullanıcı_Id'] = [tweet.user.id for tweet in tweets]  
    DataSet['Kullanıcı_Adı'] = [tweet.user.name for tweet in tweets]       
    DataSet['Dil'] = [tweet.lang for tweet in tweets]
    
    """"
    DataSet['userCreateDt'] = [tweet.user.created_at for tweet in tweets]
    DataSet['tweetRetweetCt'] = [tweet.retweet_count for tweet in tweets]
    DataSet['tweetFavoriteCt'] = [tweet.favorite_count for tweet in tweets]
    DataSet['tweetSource'] = [tweet.source for tweet in tweets]
    DataSet['userScreen'] = [tweet.user.screen_name for tweet in tweets]
    DataSet['userDesc'] = [tweet.user.description for tweet in tweets]
    DataSet['userFollowerCt'] = [tweet.user.followers_count for tweet in tweets]
    DataSet['userFriendsCt'] = [tweet.user.friends_count for tweet in tweets]
    DataSet['userLocation'] = [tweet.user.location for tweet in tweets]
    DataSet['userTimezone'] = [tweet.user.time_zone for tweet in tweets]
    DataSet['Coordinates'] = [tweet.coordinates for tweet in tweets]
    DataSet['GeoEnabled'] = [tweet.user.geo_enabled for tweet in tweets] 
    users_retweeted = []    
    for tweet in tweets:
        if tweet.place:
            tweets_place.append(tweet.place.full_name)
        else:
            tweets_place.append('null')
    
    DataSet['TweetPlace'] = [i for i in tweets_place]
    DataSet['UserWhoRetweeted'] = [i for i in users_retweeted]
    """
    print ("Veriler Kayıt Edildi!")
    return DataSet

In [290]:
if (not api):
    print ("Bağlantı Yapılamadı!")
    sys.exit(-1)
else:
    print ("Bağlantı Yapıldı!")
    
    aranan=input("Aranacak Kelimeyi Girin: ")
    
    print ("Kelimeye Uygun Tweetler Sorgulanıyor...")
    
    cursor = tweepy.Cursor(api.search_tweets, q=aranan, result_type="new", geocode="55.0000,4.0000,1000km",lang='tr')
    
    results=[]
    for item in cursor.items(10): # Remove the limit to 1000
            results.append(item)
            
    print ("Kelimeye Uygun Tweetler Sorgulandı!")
    
    DataSet = toDataFrame(results)  
    TweetTemizle(DataSet)
    
    DataSet.to_csv('tw.csv',index=False)

Bağlantı Yapıldı!
Kelimeye Uygun Tweetler Sorgulanıyor...
Kelimeye Uygun Tweetler Sorgulandı!
Veriler Kayıt Ediliyor...
Veriler Kayıt Edildi!
Tweetler Temizleniyor...
Tweetler Temizlendi!
Tweetler Çeviriliyor...
@HABERTURKTV
Cem Bey speaks like a tour of a black sea flower.… Https://t.co/odrx7hmy0a
@HABERTURKTV
Mr. Cem, who are you talking very meaningful?
@HABERTURKTV
Şükrü Bey, tolerate the growth.
@Dailyislamist cem start from the houses and let's see the back
@HABERTURKTV
Cem Bey, Şükrü Bey's 9500 dollars described and exemplified… https://t.co/osr3upıhwx
@Misafir @orsatramola sound recording and accepted indictment is enough?
@yurttastv @barisatay to the type ..... cem tried the houses
@tanjuozcanchp Cem Houses If the CHP vote warehouse is not a vote for 80 years, let's see if you give suggestions to close there.
Why don't Cem Houses? https://t.co/gnsmxyexI8
@say_cem The perveces regard their perversions to Islam, to Islam, to God
Those Trolller/Müzdeler who are not even human bei

In [291]:
Veri=pd.read_csv('tw.csv')

Veri.head()

,Tweet_Id,Tweet,Tweet_Degistirilmis,Tweet_Ceviri,Tweet_Tarih,Kullanıcı_Id,Kullanıcı_Adı,Dil
0,1601310949540888576,"@HaberturkTV @cemkucuk55 @SerapBelet @kursadoguz Merhaba \nCem bey,tam bir Karadenizin Tursunu gibi konuşuyor çiçek.… https://t.co/oDrx7HMY0A",Merhaba Cem bey tam bir Karadenizin Tursunu gibi konuşuyor çiçek,@HABERTURKTV\nCem Bey speaks like a tour of a black sea flower.… Https://t.co/odrx7hmy0a,2022-12-09 20:20:55+00:00,718092899,Fedai Ceylan,tr
1,1601304001902886912,@HaberturkTV @cemkucuk55 @SerapBelet @kursadoguz Merhaba \nCem bey çok manalı konuşuyorsun kime?.Ersan bey hiç oralı… https://t.co/ASXMR2OGqO,Merhaba Cem bey çok manalı konuşuyorsun kime Ersan bey hiç oralı,"@HABERTURKTV\nMr. Cem, who are you talking very meaningful?",2022-12-09 19:53:19+00:00,718092899,Fedai Ceylan,tr
2,1601296368056041472,"@HaberturkTV @cemkucuk55 @SerapBelet @kursadoguz Merhaba \nŞükrü bey,büyümeyi tahammül edemiyor.Sol cenahlı.Cem bey,… https://t.co/rYsO1jVDRp",Merhaba Şükrü bey büyümeyi tahammül edemiyor Sol cenahlı Cem bey,"@HABERTURKTV\nŞükrü Bey, tolerate the growth.",2022-12-09 19:22:59+00:00,718092899,Fedai Ceylan,tr
3,1601296039423938562,@dailyislamist Cem evlerinden başlasın da görelim arkayi,Cem evlerinden başlasın da görelim arkayi,@Dailyislamist cem start from the houses and let's see the back,2022-12-09 19:21:41+00:00,1461328348735549442,ORD Prof.DR Andelib,tr
4,1601293075158290433,"@HaberturkTV @cemkucuk55 @SerapBelet @kursadoguz Merhaba \nCem bey,şükrü beyin anlattığı ve örneklediği 9500 doları… https://t.co/osr3UpIhwX",Merhaba Cem bey şükrü beyin anlattığı ve örneklediği doları,"@HABERTURKTV\nCem Bey, Şükrü Bey's 9500 dollars described and exemplified… https://t.co/osr3upıhwx",2022-12-09 19:09:54+00:00,718092899,Fedai Ceylan,tr


In [292]:
Veri.tail()

,Tweet_Id,Tweet,Tweet_Degistirilmis,Tweet_Ceviri,Tweet_Tarih,Kullanıcı_Id,Kullanıcı_Adı,Dil
5,1601283708883197952,@Misafir @Orsatramola Ses kaydı ve kabul edilmiş iddianame yeterli mi?,Ses kaydı ve kabul edilmiş iddianame yeterli mi,@Misafir @orsatramola sound recording and accepted indictment is enough?,2022-12-09 18:32:41+00:00,105546239,cem özdemir,tr
6,1601265753352974337,@yurttastv @barisatay Tipine..... Cem evlerini denemiş,Tipine Cem evlerini denemiş,@yurttastv @barisatay to the type ..... cem tried the houses,2022-12-09 17:21:20+00:00,1461328348735549442,ORD Prof.DR Andelib,tr
7,1601259298419449856,@tanjuozcanchp Cem evleri 80 yıldır CHP OY DEPOSU DEGİLMİ ORALARIDA KAPAMAK İÇİN ÖNERİ VERSE GÖRELİM YIGITLIGINI YERSE 😉,Cem evleri yıldır CHP OY DEPOSU DEGİLMİ ORALARIDA KAPAMAK İÇİN ÖNERİ VERSE GÖRELİM YIGITLIGINI YERSE 😉,"@tanjuozcanchp Cem Houses If the CHP vote warehouse is not a vote for 80 years, let's see if you give suggestions to close there.",2022-12-09 16:55:41+00:00,1461328348735549442,ORD Prof.DR Andelib,tr
8,1601258845212319746,Cem evleri niye yok ? https://t.co/gNsmXyExI8,Cem evleri niye yok,Why don't Cem Houses? https://t.co/gnsmxyexI8,2022-12-09 16:53:53+00:00,1461328348735549442,ORD Prof.DR Andelib,tr
9,1601258608377098240,@say_cem Sapkınlar sapıklıklarını İslama Müslümanlığa yani Allah’a addediyorlar \nİnsan dahi olmayan şu trolller/Müsveddeler 😡😤,Sapkınlar sapıklıklarını İslama Müslümanlığa yani Allah a addediyorlar İnsan dahi olmayan şu trolller Müsveddeler 😡😤,"@say_cem The perveces regard their perversions to Islam, to Islam, to God\nThose Trolller/Müzdeler who are not even human beings 😡😤",2022-12-09 16:52:56+00:00,998255078724300800,Nurten Nurten,tr


In [293]:
KelimeAyristirma(Veri)


In [294]:
savedTweets = Veri
savedTweets['Subjectivity'] = savedTweets['Tweet_Ceviri'].apply(getSubjectivity)
savedTweets['Polarity'] = savedTweets['Tweet_Ceviri'].apply(getPolarity)
savedTweets.to_csv('duygu.csv')
savedTweets.head()

,Tweet_Id,Tweet,Tweet_Degistirilmis,Tweet_Ceviri,Tweet_Tarih,Kullanıcı_Id,Kullanıcı_Adı,Dil,Subjectivity,Polarity
0,1601310949540888576,"@HaberturkTV @cemkucuk55 @SerapBelet @kursadoguz Merhaba \nCem bey,tam bir Karadenizin Tursunu gibi konuşuyor çiçek.… https://t.co/oDrx7HMY0A",Merhaba Cem bey tam bir Karadenizin Tursunu gibi konuşuyor çiçek,@HABERTURKTV\nCem Bey speaks like a tour of a black sea flower.… Https://t.co/odrx7hmy0a,2022-12-09 20:20:55+00:00,718092899,Fedai Ceylan,tr,0.433333,-0.166667
1,1601304001902886912,@HaberturkTV @cemkucuk55 @SerapBelet @kursadoguz Merhaba \nCem bey çok manalı konuşuyorsun kime?.Ersan bey hiç oralı… https://t.co/ASXMR2OGqO,Merhaba Cem bey çok manalı konuşuyorsun kime Ersan bey hiç oralı,"@HABERTURKTV\nMr. Cem, who are you talking very meaningful?",2022-12-09 19:53:19+00:00,718092899,Fedai Ceylan,tr,0.650000,0.650000
2,1601296368056041472,"@HaberturkTV @cemkucuk55 @SerapBelet @kursadoguz Merhaba \nŞükrü bey,büyümeyi tahammül edemiyor.Sol cenahlı.Cem bey,… https://t.co/rYsO1jVDRp",Merhaba Şükrü bey büyümeyi tahammül edemiyor Sol cenahlı Cem bey,"@HABERTURKTV\nŞükrü Bey, tolerate the growth.",2022-12-09 19:22:59+00:00,718092899,Fedai Ceylan,tr,0.000000,0.000000
3,1601296039423938562,@dailyislamist Cem evlerinden başlasın da görelim arkayi,Cem evlerinden başlasın da görelim arkayi,@Dailyislamist cem start from the houses and let's see the back,2022-12-09 19:21:41+00:00,1461328348735549442,ORD Prof.DR Andelib,tr,0.000000,0.000000
4,1601293075158290433,"@HaberturkTV @cemkucuk55 @SerapBelet @kursadoguz Merhaba \nCem bey,şükrü beyin anlattığı ve örneklediği 9500 doları… https://t.co/osr3UpIhwX",Merhaba Cem bey şükrü beyin anlattığı ve örneklediği doları,"@HABERTURKTV\nCem Bey, Şükrü Bey's 9500 dollars described and exemplified… https://t.co/osr3upıhwx",2022-12-09 19:09:54+00:00,718092899,Fedai Ceylan,tr,0.000000,0.000000


In [295]:
savedTweets.tail()

,Tweet_Id,Tweet,Tweet_Degistirilmis,Tweet_Ceviri,Tweet_Tarih,Kullanıcı_Id,Kullanıcı_Adı,Dil,Subjectivity,Polarity
5,1601283708883197952,@Misafir @Orsatramola Ses kaydı ve kabul edilmiş iddianame yeterli mi?,Ses kaydı ve kabul edilmiş iddianame yeterli mi,@Misafir @orsatramola sound recording and accepted indictment is enough?,2022-12-09 18:32:41+00:00,105546239,cem özdemir,tr,0.45,0.2
6,1601265753352974337,@yurttastv @barisatay Tipine..... Cem evlerini denemiş,Tipine Cem evlerini denemiş,@yurttastv @barisatay to the type ..... cem tried the houses,2022-12-09 17:21:20+00:00,1461328348735549442,ORD Prof.DR Andelib,tr,0.00,0.0
7,1601259298419449856,@tanjuozcanchp Cem evleri 80 yıldır CHP OY DEPOSU DEGİLMİ ORALARIDA KAPAMAK İÇİN ÖNERİ VERSE GÖRELİM YIGITLIGINI YERSE 😉,Cem evleri yıldır CHP OY DEPOSU DEGİLMİ ORALARIDA KAPAMAK İÇİN ÖNERİ VERSE GÖRELİM YIGITLIGINI YERSE 😉,"@tanjuozcanchp Cem Houses If the CHP vote warehouse is not a vote for 80 years, let's see if you give suggestions to close there.",2022-12-09 16:55:41+00:00,1461328348735549442,ORD Prof.DR Andelib,tr,0.00,0.0
8,1601258845212319746,Cem evleri niye yok ? https://t.co/gNsmXyExI8,Cem evleri niye yok,Why don't Cem Houses? https://t.co/gnsmxyexI8,2022-12-09 16:53:53+00:00,1461328348735549442,ORD Prof.DR Andelib,tr,0.00,0.0
9,1601258608377098240,@say_cem Sapkınlar sapıklıklarını İslama Müslümanlığa yani Allah’a addediyorlar \nİnsan dahi olmayan şu trolller/Müsveddeler 😡😤,Sapkınlar sapıklıklarını İslama Müslümanlığa yani Allah a addediyorlar İnsan dahi olmayan şu trolller Müsveddeler 😡😤,"@say_cem The perveces regard their perversions to Islam, to Islam, to God\nThose Trolller/Müzdeler who are not even human beings 😡😤",2022-12-09 16:52:56+00:00,998255078724300800,Nurten Nurten,tr,0.10,0.0
